In [1]:
import pandas as pd

In [15]:
gold_df = pd.read_csv('gold_dataset.csv', sep='\t', on_bad_lines='skip')

In [16]:
gold_df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'source', 'annot'], dtype='object')

In [18]:
auto_df = pd.read_csv('fifth_part.csv', sep='\t', on_bad_lines='skip')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc4 in position 28: invalid continuation byte